In [1]:
import tensorflow as tf

/Users/Linh-Main/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
data = tf.placeholder(tf.float32, [None, 784])
labels = tf.placeholder(tf.float32, [None, 10])
phase = tf.placeholder(tf.bool,name = 'phase')
input_layer = tf.reshape(data, [-1, 28, 28, 1])

def conv_pool(inputs,kernel_size,name):
    in_shape = kernel_size[:2]
    in_channels = kernel_size[2]
    out_channels = kernel_size[3]
    kernels = tf.get_variable('cw'+name,[5,5,in_channels,out_channels],
                              initializer = tf.truncated_normal_initializer())
    biases = tf.get_variable('cb'+name, [in_shape[0],in_shape[1],out_channels],dtype =tf.float32,
                            initializer = tf.zeros_initializer())
    conv = tf.nn.conv2d(input = inputs,
                            filter = kernels,
                            strides=[1,1,1,1],
                           padding = 'SAME')
    conv = tf.nn.relu(conv+biases)

    pool  =  tf.nn.max_pool(value = conv, 
                           ksize=[1,2,2,1], 
                           strides=[1,2,2,1],
                          padding = 'SAME')
    return pool



def ff(flat, weights_list, sizes, train):
    counter = 0
    for weights,size in zip(weights_list[:-1], sizes[1:-1]):
        if train == True:
            weights_adj = tf.nn.dropout(weights, 0.6)
        else:
            weights_adj = weights*0.6
        flat = tf.matmul(flat, weights_adj)
        mean, var = tf.nn.moments(flat, [0])
        offset = tf.Variable(tf.zeros(size), name='offset{}'.format(counter))
        scale = tf.Variable(tf.ones(size), name='scale{}'.format(counter))
        flat_BN = tf.nn.batch_normalization(flat, mean, var, offset, scale, 0.01)
        flat = tf.nn.relu(flat_BN)
        counter += 1
    if train == True:
        weights_adj = tf.nn.dropout(weights, 0.6)
    else:
        weights_adj = weights*0.5
    logits = tf.matmul(flat, weights_list[-1])
    return logits

In [4]:
conv1 = conv_pool(input_layer,[28,28,1,8],'conv1')
conv2 = conv_pool(conv1,[14,14,8,16],'conv2')
shape2 = 7*7*16

sizes = [shape2,200,100,10]
flat = tf.reshape(conv2,[-1,shape2])
weight_list = []
counter = 1
for in_size,out_size in zip(sizes[:-1],sizes[1:]):
    name = str(counter)
    weights = tf.get_variable('dw'+name,[in_size,out_size],
                                   initializer = tf.truncated_normal_initializer(stddev = in_size**(-0.5)))
    weight_list.append(weights)
    counter += 1

logits = ff(flat,weight_list,sizes,train = True)    
cross_entropy = tf.losses.softmax_cross_entropy(onehot_labels = labels, 
                                                logits = logits) 

# define evaluation metrics
logits_wo_drop = ff(flat,weight_list,sizes, train = False)    

correct_prediction = tf.equal(tf.argmax(logits_wo_drop,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [5]:
def train():
    # optimizer
    import sys
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    optimize = optimizer.minimize(cross_entropy)
    
    epochs = 10
    batch_size = 60
    n_batches = int(len(mnist.train.images)/batch_size)
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()

    # create saver
    saver = tf.train.Saver()

    predictions_o = []
    for epoch in range(epochs):
        avg_cost =0
        for i in range(n_batches):
            inputs_batch, labels_batch = mnist.train.next_batch(batch_size)
            _,cost = sess.run([optimize,cross_entropy],feed_dict={data: inputs_batch, labels: labels_batch})
            avg_cost += cost/n_batches
            sys.stdout.write('\r batch cost {}'.format(cost))
        test_acc = sess.run(accuracy, feed_dict={data: mnist.test.images, labels:mnist.test.labels})
        print("\n Epoch:", (epoch + 1), "cost =", "{:.4f}".format(avg_cost), "test accuracy: {:.4f}".format(test_acc))


    print("\nTraining complete!")
    

In [6]:
%%time
train()

 batch cost 0.0170771740376949356
 Epoch: 1 cost = 0.1348 test accuracy: 0.9857
 batch cost 0.0642288103699684155
 Epoch: 2 cost = 0.0652 test accuracy: 0.9884
 batch cost 0.0229990649968385757
 Epoch: 3 cost = 0.0486 test accuracy: 0.9897
 batch cost 0.02516917139291763344
 Epoch: 4 cost = 0.0419 test accuracy: 0.9897
 batch cost 0.01175931748002767666
 Epoch: 5 cost = 0.0381 test accuracy: 0.9899
 batch cost 0.03820027783513069735
 Epoch: 6 cost = 0.0345 test accuracy: 0.9887
 batch cost 0.09632960706949234555
 Epoch: 7 cost = 0.0295 test accuracy: 0.9910
 batch cost 0.12816590070724487974
 Epoch: 8 cost = 0.0295 test accuracy: 0.9904
 batch cost 0.00059497467009350664
 Epoch: 9 cost = 0.0268 test accuracy: 0.9909
 batch cost 0.00187772826757282028
 Epoch: 10 cost = 0.0251 test accuracy: 0.9904

Training complete!
CPU times: user 23min 22s, sys: 4min, total: 27min 23s
Wall time: 13min
